In [ ]:
pip install ESRNN

In [ ]:
from ESRNN.m4_data import prepare_m4_data
from ESRNN.utils_evaluation import evaluate_prediction_owa
from ESRNN import ESRNN
import os
import sys
import random
import numpy as np
import random
import string
from datetime import datetime
from sktime.forecasting.naive import NaiveForecaster
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, pacf
import math
from pandas import ExcelWriter
from pandas import ExcelFile
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import smape_loss
from sktime.performance_metrics.forecasting import mase_loss
from sktime.utils.plotting.forecasting import plot_ys
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
from google.colab import files
#Use to import io, which opens the file from the Colab drive
import io
uploaded = files.upload()

Saving DATA.xlsx to DATA.xlsx


In [ ]:
data1 = pd.read_excel('DATA.xlsx', sheet_name=2)
data1.head()

In [ ]:
# Need to transform into csv file in order to input the data in the ESRNN algorithm
data1.to_csv('data_return.csv', encoding='utf-8',index=False)
data1 = pd.read_csv('data.csv')
data1.head()

In [ ]:
features_considered = ['Date','Russeltop200','MSCIJapan','SP500IT','SP500HealthCare','SP500Industrial']
series = data1[features_considered]
series.head()

In [ ]:
# Manipulate the date
testsize = 0.2

y1 = series['Russeltop200']
y2 = series['MSCIJapan']
y3 = series['SP500IT']
y4 = series['SP500HealthCare']
y5 = series['SP500Industrial']
y_Date = series ['Date']
y_Date2 = pd.date_range(end = y_Date.iloc[-1], periods = len(y_Date),freq='D')
fhlength = math.floor(len(y1) * testsize)

y1_train, y1_test = temporal_train_test_split(y1, test_size=fhlength)
#plot_ys(y1_train, y1_test, labels=["y1_train", "y1_test"])
print(y1_train.shape[0], y1_test.shape[0])

y2_train, y2_test = temporal_train_test_split(y2, test_size=fhlength)
#plot_ys(y2_train, y2_test, labels=["y2_train", "y2_test"])
print(y2_train.shape[0], y2_test.shape[0])

y3_train, y3_test = temporal_train_test_split(y3, test_size=fhlength)
#plot_ys(y3_train, y3_test, labels=["y3_train", "y3_test"])
print(y3_train.shape[0], y3_test.shape[0])

y4_train, y4_test = temporal_train_test_split(y4, test_size=fhlength)
#plot_ys(y4_train, y4_test, labels=["y4_train", "y4_test"])
print(y4_train.shape[0], y4_test.shape[0])

y5_train, y5_test = temporal_train_test_split(y5, test_size=fhlength)
#plot_ys(y5_train, y5_test, labels=["y5_train", "y5_test"])
print(y5_train.shape[0], y5_test.shape[0])

Date_train, Date_test = temporal_train_test_split(y_Date2, test_size=fhlength)

# Set Forecasting interval:
fh = np.arange(len(y1_test)) + 1
fh

In [ ]:
forecaster = NaiveForecaster(strategy="seasonal_last",sp=252)


forecaster.fit(y1_train)
y1_pred_naive = forecaster.predict(fh)
#plot_ys(y1_train, y1_test, y1_pred_naive, labels=["y1_train", "y1_test", "y1_pred_naive"])

forecaster.fit(y2_train)
y2_pred_naive = forecaster.predict(fh)
#plot_ys(y2_train, y2_test, y2_pred_naive, labels=["y2_train", "y2_test", "y2_pred_naive"])

forecaster.fit(y3_train)
y3_pred_naive = forecaster.predict(fh)
#plot_ys(y3_train, y3_test, y3_pred_naive, labels=["y3_train", "y3_test", "y3_pred_naive"])

forecaster.fit(y4_train)
y4_pred_naive = forecaster.predict(fh)
#plot_ys(y4_train, y4_test, y4_pred_naive, labels=["y4_train", "y4_test", "y4_pred_naive"])

forecaster.fit(y5_train)
y5_pred_naive = forecaster.predict(fh)


In [ ]:
# Create large dataset list
identity = []
name = ['Russeltop200','MSCIJapan','SP500IT','SP500HealthCare','SP500Industrial']
name_list1 = []
name_list2 = []
for item in name:
    for i in range(len(y1_train)):
        name_list1.append(item)

for item in name:
    for i in range(len(y1_test)):
        name_list2.append(item)
y_train_list = list(y1_train) + list(y2_train) + list(y3_train) + list(y4_train) + list(y5_train)
y_test_list = list(y1_test) + list(y2_test) + list(y3_test) + list(y4_test) + list(y5_test)
y_hat_naive_list = list(y1_pred_naive) + list(y2_pred_naive) + list(y3_pred_naive) + list(y4_pred_naive) + list(y5_pred_naive)
len(y_hat_naive_list)

In [ ]:
# Combine data
data1 = {'unique_id':name_list1,'ds':list(Date_train) *5, 'x':['Macro']*len(Date_train) *5} 
data2 = {'unique_id':name_list1,'ds':list(Date_train) *5, 'y':y_train_list} 
data3 = {'unique_id':name_list2, 'ds':list(Date_test) *5 ,'x':['Macro']*len(Date_test)*5} 
data4 = {'unique_id':name_list2,'ds':list(Date_test) *5, 'y':y_test_list,'y_hat_naive2': y_hat_naive_list} 

X1_train_df = pd.DataFrame(data1)

y1_train_df = pd.DataFrame(data2)

X1_test_df = pd.DataFrame(data3)

y1_test_df = pd.DataFrame(data4)

X1_test_df = X1_test_df.reset_index(drop = True)
y1_test_df = y1_test_df.reset_index(drop = True)

In [ ]:
total = 999999
learning_rate = 0
learning_multiplier = 0
learning_decay = 0
level_penalty = 0
input = 0
steps = 0
count = 0
def tune_ESRNN(lr, lrm, step, lerd, penalty, input, total):
    model = ESRNN(max_epochs=30, freq_of_test=3, batch_size=4, learning_rate=lr,
              per_series_lr_multip=lrm, lr_scheduler_step_size=step,
              lr_decay=lerd, gradient_clipping_threshold=20,
              rnn_weight_decay=0.0, level_variability_penalty=penalty,
              testing_percentile=51, training_percentile=49,
              ensemble=False, max_periods=12, seasonality=[7],
              input_size=input, output_size=1225,
              cell_type='LSTM', state_hsize=50,
              dilations=[[1],[7],[14]], add_nl_layer=False,
              random_seed=1, device='cuda')
    model.fit(X1_train_df, y1_train_df, X1_test_df, y1_test_df)
    y_hat_df = model.predict(X1_test_df)
    final_owa, final_mase, final_smape = evaluate_prediction_owa(y_hat_df, y1_train_df,
                                                             X1_test_df, y1_test_df,
                                                             naive2_seasonality=7)
    final = (final_owa + final_mase + final_smape)/3
    diff1 = abs(final_owa - final_mase)
    diff2 = abs(final_mase - final_smape)
    diff3 = abs(final_smape - final_owa)
  #print("Model " + str(count) + " complete!")
  
    if final <= total and diff1 <=2 and diff2 <= 2 and diff3 <= 2:
        learning_rate = lr
        learning_multiplier = lrm
        learning_decay = lerd
        level_penalty = penalty
        steps = step
        total = final
    return [learning_rate, learning_multiplier,learning_decay, penalty, steps, total]


In [ ]:
# Parameters' tuning range
learning_rates = [0.0007, 0.0003, 0.0001]
learning_multipliers = [0.2,0.3,0.5]
learning_decays = [0.2,0.5]
level_penaltys = [50,100]
inputs = [10,15,20,40]
steps = [12,15,20]

In [ ]:
# Start the tuning process
for i in learning_rates:
    for k in learning_multipliers:
        for j in learning_decays:
            for h in level_penaltys:
                for g in inputs:
                    for l in steps:
                        Result = tune_ESRNN(i, k, l, j, h, g, total)
                        print("Model " + str(count) + " complete!")
                        count = count + 1


In [ ]:
# Show the best hyperparameters
Result

In [ ]:
# Run the best model
model = ESRNN(max_epochs=30, freq_of_test=3, batch_size=4, learning_rate=7e-4,
              per_series_lr_multip=0.2, lr_scheduler_step_size=12,
              lr_decay=0.5, gradient_clipping_threshold=20,
              rnn_weight_decay=0.0, level_variability_penalty=100,
              testing_percentile=51, training_percentile=49,
              ensemble=False, max_periods=12, seasonality=[7],
              input_size=40, output_size=1225,
              cell_type='LSTM', state_hsize=50,
              dilations=[[1],[7],[14]], add_nl_layer=False,
              random_seed=1, device='cuda') # Can use either cpu or cuda(GPU)

In [ ]:
# Fit model
# If y_test_df is provided the model
# will evaluate predictions on
# this set every freq_test epochs
model.fit(X1_train_df, y1_train_df, X1_test_df, y1_test_df)


In [ ]:
# Evaluation of the result
final_owa, final_mase, final_smape = evaluate_prediction_owa(y_hat_df, y1_train_df,
                                                             X1_test_df, y1_test_df,
                                                             naive2_seasonality=7)

In [ ]:
# Write the entire prediction into csv file
y_hat_df.to_csv('ESRNN_output.csv')

In [ ]:
# Separate the result
y_hat_df_R2 = y_hat_df[y_hat_df['unique_id'] == 'Russeltop200'].set_index(y1_test.index)
y_hat_df_MSCI = y_hat_df[y_hat_df['unique_id'] == 'MSCIJapan'].set_index(y2_test.index)
y_hat_df_Health = y_hat_df[y_hat_df['unique_id'] == 'SP500HealthCare'].set_index(y3_test.index)
y_hat_df_It = y_hat_df[y_hat_df['unique_id'] == 'SP500IT'].set_index(y4_test.index)
y_hat_df_Indus = y_hat_df[y_hat_df['unique_id'] == 'SP500Industrial'].set_index(y5_test.index)

In [ ]:
# Plot the result
plot_ys(y1_train, y1_test, y_hat_df_R2.y_hat, labels=["y1_train", "y1_test", "y1_pred_ESRNN"])
plot_ys(y2_train, y2_test, y_hat_df_MSCI.y_hat, labels=["y2_train", "y2_test", "y2_pred_ESRNN"])
plot_ys(y3_train, y3_test, y_hat_df_Health.y_hat, labels=["y3_train", "y3_test", "y3_pred_ESRNN"])
plot_ys(y4_train, y4_test, y_hat_df_It.y_hat, labels=["y4_train", "y4_test", "y4_pred_ESRNN"])
plot_ys(y5_train, y5_test, y_hat_df_Indus.y_hat, labels=["y5_train", "y5_test", "y5_pred_ESRNN"])

In [ ]:
# Read in the risk free csv, and change it into a list
data_rf = pd.read_csv("risk free.csv")
rf = list(data_rf.rf)
rf

In [ ]:
# Calculate return and put it and the original prediction into two csv file
data1 = {'Russeltop200': y_hat_df_R2.y_hat -1 + rf,'MSCIJapan': y_hat_df_MSCI.y_hat -1 + rf,'SP500IT':y_hat_df_It.y_hat - 1 + rf,'SP500HealthCare':y_hat_df_Health.y_hat -1 + rf,'SP500Industrial': y_hat_df_Indus.y_hat -1 + rf}
data2 = {'Russeltop200': y_hat_df_R2.y_hat,'MSCIJapan': y_hat_df_MSCI.y_hat ,'SP500IT':y_hat_df_It.y_hat ,'SP500HealthCare':y_hat_df_Health.y_hat ,'SP500Industrial': y_hat_df_Indus.y_hat }
df_return = pd.DataFrame(data1)
df_premium = pd.DataFrame(data2)
df_return.insert(0, "Date",Date_test)
df_premium.insert(0, "Date",Date_test)
df_return = df_return.reset_index(drop = True)
df_premium = df_premium.reset_index(drop = True)
df_premium.head()

In [ ]:
df_return.to_csv('prediction(return)_output.csv') 
df_premium.to_csv('prediction(premium)_output.csv') 